In [10]:
import pandas as pd
import numpy as np
path = '../data'
y_train = pd.read_csv(f'{path}/y_train.csv', index_col = 'key_value')

In [2]:
train = pd.read_pickle('../data/train_features1320_score_0.85577.pkl')
test = pd.read_pickle('../data/test_features1320_score_0.85577.pkl')

In [24]:
aux = (train.isna().sum()/len(train)).sort_values()
features_without_na = aux[aux==0]
print(f'feature sin missing values {len(features_without_na)} representando el {round(len(features_without_na)/len(train.columns), 3)} de features totales')
list_prec_na = [(0,0.1), (0.1,0.2), (0.2,0.3), (0.3,0.4), (0.4,0.5), (0.5,0.6), (0.6,0.7), (0.7,0.8), (0.8,0.9),(0.9,1.1)]
for i in list_prec_na:
    features_with_na = aux[(aux>=i[0]) & (aux<i[1])]
    print(f'features con porc de missing values de {i[0]} a {i[1]} : {len(features_with_na)} representando el {round(len(features_with_na)/len(train.columns), 3)} de features totales')

feature sin missing values 124 representando el 0.094 de features totales
features con porc de missing values de 0 a 0.1 : 321 representando el 0.243 de features totales
features con porc de missing values de 0.1 a 0.2 : 11 representando el 0.008 de features totales
features con porc de missing values de 0.2 a 0.3 : 1 representando el 0.001 de features totales
features con porc de missing values de 0.3 a 0.4 : 29 representando el 0.022 de features totales
features con porc de missing values de 0.4 a 0.5 : 190 representando el 0.144 de features totales
features con porc de missing values de 0.5 a 0.6 : 84 representando el 0.064 de features totales
features con porc de missing values de 0.6 a 0.7 : 26 representando el 0.02 de features totales
features con porc de missing values de 0.7 a 0.8 : 53 representando el 0.04 de features totales
features con porc de missing values de 0.8 a 0.9 : 186 representando el 0.141 de features totales
features con porc de missing values de 0.9 a 1.1 : 419 

In [25]:
0.094 + 0.243 + 0.008 + 0.001 + 0.022 + 0.144 + 0.064 + 0.02 + 0.04 + 0.141 + 0.317

1.094

In [8]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

def cross_validation_lightgbm(train, y_train, test):
    folds = [train.index[t] for t, v in KFold(5).split(train)]
    test_probs = []
    train_probs = []
    fi = []
    for i, idx in enumerate(folds):
        print("*"*10, i, "*"*10)
        Xt = train.loc[idx]
        yt = y_train.loc[Xt.index].target

        Xv = train.drop(Xt.index)
        yv = y_train.loc[Xv.index].target

        learner = LGBMClassifier(n_estimators=1000, boosting_type='gbdt',min_child_samples=1500, 
                       colsample_bytree=0.8,subsample=0.8, max_bin=200, learning_rate=0.1, random_state=42)
        learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                    eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
        test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
        train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
        fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

    test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
    train_probs = pd.concat(train_probs)
    fi = pd.concat(fi, axis=1).mean(axis=1)
    print("*" * 21)
    print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
    print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))])) 
    return test_probs, fi

In [34]:
[test_probs,fi] = cross_validation_lightgbm(train, y_train, test)

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852624	training's binary_logloss: 0.296118	valid_1's auc: 0.843978	valid_1's binary_logloss: 0.302702
[100]	training's auc: 0.865395	training's binary_logloss: 0.285035	valid_1's auc: 0.851313	valid_1's binary_logloss: 0.296541
[150]	training's auc: 0.8731	training's binary_logloss: 0.278557	valid_1's auc: 0.853176	valid_1's binary_logloss: 0.29491
[200]	training's auc: 0.879572	training's binary_logloss: 0.273373	valid_1's auc: 0.854061	valid_1's binary_logloss: 0.294174
[250]	training's auc: 0.885087	training's binary_logloss: 0.26889	valid_1's auc: 0.854427	valid_1's binary_logloss: 0.293868
[300]	training's auc: 0.890454	training's binary_logloss: 0.264554	valid_1's auc: 0.854846	valid_1's binary_logloss: 0.293555
Early stopping, best iteration is:
[291]	training's auc: 0.889494	training's binary_logloss: 0.265343	valid_1's auc: 0.854926	valid_1's binary_logloss: 0.293491
***

In [9]:
def get_feature_selection(fi):
    zero_importance = fi[fi==0]
    aux = fi[fi>0].sort_values(ascending=False)
    keep_columns = []
    count = 0
    for feature,values in zip(aux.index, aux.values):
        count+=values
        if count<=0.99:
            keep_columns.append(feature)
            
    print(f'total de variables : {len(fi)}')
    print(f'variables con importancia acumulada al 99% : {len(keep_columns)}')
    print(f'variables con zero importancia : {len(zero_importance)}')
    return keep_columns, zero_importance

In [37]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 1320
variables con importancia acumulada al 99% : 1243
variables con zero importancia : 0


In [40]:
keep_columns_v2 = keep_columns

In [39]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852378	training's binary_logloss: 0.296174	valid_1's auc: 0.843472	valid_1's binary_logloss: 0.303029
[100]	training's auc: 0.865188	training's binary_logloss: 0.285052	valid_1's auc: 0.851167	valid_1's binary_logloss: 0.296636
[150]	training's auc: 0.873043	training's binary_logloss: 0.278564	valid_1's auc: 0.853217	valid_1's binary_logloss: 0.294906
[200]	training's auc: 0.879468	training's binary_logloss: 0.273405	valid_1's auc: 0.854177	valid_1's binary_logloss: 0.294115
[250]	training's auc: 0.885122	training's binary_logloss: 0.268884	valid_1's auc: 0.854652	valid_1's binary_logloss: 0.293749
Early stopping, best iteration is:
[262]	training's auc: 0.886401	training's binary_logloss: 0.267808	valid_1's auc: 0.854745	valid_1's binary_logloss: 0.293678
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852395	training's

In [41]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 1243
variables con importancia acumulada al 99% : 1185
variables con zero importancia : 0


In [42]:
keep_columns_v3 = keep_columns

In [43]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852581	training's binary_logloss: 0.296109	valid_1's auc: 0.843226	valid_1's binary_logloss: 0.303089
[100]	training's auc: 0.865371	training's binary_logloss: 0.285	valid_1's auc: 0.850683	valid_1's binary_logloss: 0.296826
[150]	training's auc: 0.873179	training's binary_logloss: 0.278537	valid_1's auc: 0.852967	valid_1's binary_logloss: 0.294983
[200]	training's auc: 0.879609	training's binary_logloss: 0.273353	valid_1's auc: 0.853852	valid_1's binary_logloss: 0.294239
[250]	training's auc: 0.885055	training's binary_logloss: 0.268861	valid_1's auc: 0.854434	valid_1's binary_logloss: 0.293741
Early stopping, best iteration is:
[272]	training's auc: 0.88721	training's binary_logloss: 0.267039	valid_1's auc: 0.854625	valid_1's binary_logloss: 0.293598
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85237	training's bina

In [44]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 1185
variables con importancia acumulada al 99% : 1127
variables con zero importancia : 0


In [45]:
keep_columns_v4 = keep_columns

In [46]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852748	training's binary_logloss: 0.296059	valid_1's auc: 0.843539	valid_1's binary_logloss: 0.30304
[100]	training's auc: 0.865354	training's binary_logloss: 0.284976	valid_1's auc: 0.851044	valid_1's binary_logloss: 0.296813
[150]	training's auc: 0.873158	training's binary_logloss: 0.278522	valid_1's auc: 0.853108	valid_1's binary_logloss: 0.295065
[200]	training's auc: 0.879555	training's binary_logloss: 0.273359	valid_1's auc: 0.854173	valid_1's binary_logloss: 0.294225
[250]	training's auc: 0.885139	training's binary_logloss: 0.268794	valid_1's auc: 0.854656	valid_1's binary_logloss: 0.293835
Early stopping, best iteration is:
[265]	training's auc: 0.886761	training's binary_logloss: 0.2675	valid_1's auc: 0.854778	valid_1's binary_logloss: 0.293719
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852416	training's bi

In [47]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 1127
variables con importancia acumulada al 99% : 1079
variables con zero importancia : 0


In [51]:
keep_columns_v5 = keep_columns

In [55]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852639	training's binary_logloss: 0.296128	valid_1's auc: 0.843327	valid_1's binary_logloss: 0.303022
[100]	training's auc: 0.865461	training's binary_logloss: 0.285033	valid_1's auc: 0.850839	valid_1's binary_logloss: 0.296821
[150]	training's auc: 0.873425	training's binary_logloss: 0.278506	valid_1's auc: 0.852953	valid_1's binary_logloss: 0.294951
[200]	training's auc: 0.879771	training's binary_logloss: 0.273336	valid_1's auc: 0.85393	valid_1's binary_logloss: 0.29418
[250]	training's auc: 0.885369	training's binary_logloss: 0.268747	valid_1's auc: 0.854595	valid_1's binary_logloss: 0.293587
Early stopping, best iteration is:
[247]	training's auc: 0.885073	training's binary_logloss: 0.268993	valid_1's auc: 0.854636	valid_1's binary_logloss: 0.293551
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852238	training's b

In [56]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 1079
variables con importancia acumulada al 99% : 1033
variables con zero importancia : 0


In [57]:
keep_columns_v6 = keep_columns

In [58]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852468	training's binary_logloss: 0.296145	valid_1's auc: 0.843745	valid_1's binary_logloss: 0.302847
[100]	training's auc: 0.865091	training's binary_logloss: 0.285128	valid_1's auc: 0.850863	valid_1's binary_logloss: 0.296842
[150]	training's auc: 0.873151	training's binary_logloss: 0.278585	valid_1's auc: 0.853074	valid_1's binary_logloss: 0.295076
[200]	training's auc: 0.879489	training's binary_logloss: 0.273428	valid_1's auc: 0.854201	valid_1's binary_logloss: 0.294205
[250]	training's auc: 0.885268	training's binary_logloss: 0.268801	valid_1's auc: 0.854445	valid_1's binary_logloss: 0.293948
[300]	training's auc: 0.890445	training's binary_logloss: 0.26455	valid_1's auc: 0.854723	valid_1's binary_logloss: 0.293734
Early stopping, best iteration is:
[299]	training's auc: 0.890356	training's binary_logloss: 0.264623	valid_1's auc: 0.854743	valid_1's binary_logloss: 0.293721


In [59]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 1033
variables con importancia acumulada al 99% : 996
variables con zero importancia : 0


In [60]:
keep_columns_v7 = keep_columns

In [61]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852459	training's binary_logloss: 0.296164	valid_1's auc: 0.843402	valid_1's binary_logloss: 0.303
[100]	training's auc: 0.865472	training's binary_logloss: 0.285009	valid_1's auc: 0.850748	valid_1's binary_logloss: 0.29696
[150]	training's auc: 0.873124	training's binary_logloss: 0.27857	valid_1's auc: 0.852916	valid_1's binary_logloss: 0.295064
[200]	training's auc: 0.879649	training's binary_logloss: 0.273366	valid_1's auc: 0.85421	valid_1's binary_logloss: 0.294194
Early stopping, best iteration is:
[223]	training's auc: 0.882207	training's binary_logloss: 0.271223	valid_1's auc: 0.854436	valid_1's binary_logloss: 0.294009
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852466	training's binary_logloss: 0.296253	valid_1's auc: 0.844005	valid_1's binary_logloss: 0.302816
[100]	training's auc: 0.865123	training's binar

In [62]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 996
variables con importancia acumulada al 99% : 959
variables con zero importancia : 0


In [63]:
keep_columns_v8 = keep_columns  #####

In [64]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852298	training's binary_logloss: 0.296264	valid_1's auc: 0.843309	valid_1's binary_logloss: 0.303084
[100]	training's auc: 0.864997	training's binary_logloss: 0.285311	valid_1's auc: 0.850747	valid_1's binary_logloss: 0.296884
[150]	training's auc: 0.872949	training's binary_logloss: 0.278786	valid_1's auc: 0.853096	valid_1's binary_logloss: 0.29497
[200]	training's auc: 0.879372	training's binary_logloss: 0.273608	valid_1's auc: 0.853989	valid_1's binary_logloss: 0.294218
[250]	training's auc: 0.885086	training's binary_logloss: 0.268984	valid_1's auc: 0.854757	valid_1's binary_logloss: 0.293682
Early stopping, best iteration is:
[245]	training's auc: 0.884554	training's binary_logloss: 0.269439	valid_1's auc: 0.854769	valid_1's binary_logloss: 0.293692
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852452	training's 

In [65]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 959
variables con importancia acumulada al 99% : 925
variables con zero importancia : 0


In [66]:
keep_columns_v9 = keep_columns

In [67]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852516	training's binary_logloss: 0.296254	valid_1's auc: 0.843516	valid_1's binary_logloss: 0.302977
[100]	training's auc: 0.865232	training's binary_logloss: 0.285187	valid_1's auc: 0.850954	valid_1's binary_logloss: 0.296829
[150]	training's auc: 0.873078	training's binary_logloss: 0.278686	valid_1's auc: 0.852975	valid_1's binary_logloss: 0.295036
[200]	training's auc: 0.87934	training's binary_logloss: 0.273548	valid_1's auc: 0.854086	valid_1's binary_logloss: 0.294182
[250]	training's auc: 0.884858	training's binary_logloss: 0.269041	valid_1's auc: 0.854628	valid_1's binary_logloss: 0.293779
Early stopping, best iteration is:
[243]	training's auc: 0.884094	training's binary_logloss: 0.269646	valid_1's auc: 0.854675	valid_1's binary_logloss: 0.29375
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852593	training's b

In [68]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 925
variables con importancia acumulada al 99% : 893
variables con zero importancia : 0


In [69]:
keep_columns_v10 = keep_columns

In [70]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852424	training's binary_logloss: 0.296169	valid_1's auc: 0.843096	valid_1's binary_logloss: 0.303116
[100]	training's auc: 0.864997	training's binary_logloss: 0.285205	valid_1's auc: 0.850686	valid_1's binary_logloss: 0.296831
[150]	training's auc: 0.873035	training's binary_logloss: 0.278697	valid_1's auc: 0.852976	valid_1's binary_logloss: 0.294935
[200]	training's auc: 0.879225	training's binary_logloss: 0.273581	valid_1's auc: 0.854006	valid_1's binary_logloss: 0.294105
[250]	training's auc: 0.884822	training's binary_logloss: 0.269046	valid_1's auc: 0.854531	valid_1's binary_logloss: 0.293644
Early stopping, best iteration is:
[246]	training's auc: 0.884357	training's binary_logloss: 0.269387	valid_1's auc: 0.854534	valid_1's binary_logloss: 0.293652
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852476	training's

In [71]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 893
variables con importancia acumulada al 99% : 865
variables con zero importancia : 0


In [72]:
keep_columns_v11 = keep_columns

In [73]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852459	training's binary_logloss: 0.29622	valid_1's auc: 0.84364	valid_1's binary_logloss: 0.302825
[100]	training's auc: 0.865101	training's binary_logloss: 0.285226	valid_1's auc: 0.851059	valid_1's binary_logloss: 0.29658
[150]	training's auc: 0.873016	training's binary_logloss: 0.278746	valid_1's auc: 0.853326	valid_1's binary_logloss: 0.294692
[200]	training's auc: 0.879333	training's binary_logloss: 0.273557	valid_1's auc: 0.85455	valid_1's binary_logloss: 0.293729
Early stopping, best iteration is:
[222]	training's auc: 0.881816	training's binary_logloss: 0.27154	valid_1's auc: 0.85479	valid_1's binary_logloss: 0.293545
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852111	training's binary_logloss: 0.296389	valid_1's auc: 0.844089	valid_1's binary_logloss: 0.30262
[100]	training's auc: 0.864938	training's binary

In [74]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 865
variables con importancia acumulada al 99% : 836
variables con zero importancia : 0


In [75]:
keep_columns_v12 = keep_columns

In [76]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852689	training's binary_logloss: 0.29612	valid_1's auc: 0.843463	valid_1's binary_logloss: 0.303065
[100]	training's auc: 0.865251	training's binary_logloss: 0.285103	valid_1's auc: 0.85076	valid_1's binary_logloss: 0.296818
[150]	training's auc: 0.873026	training's binary_logloss: 0.278704	valid_1's auc: 0.852905	valid_1's binary_logloss: 0.295086
[200]	training's auc: 0.879429	training's binary_logloss: 0.273537	valid_1's auc: 0.853971	valid_1's binary_logloss: 0.294201
[250]	training's auc: 0.885144	training's binary_logloss: 0.268918	valid_1's auc: 0.854477	valid_1's binary_logloss: 0.293818
Early stopping, best iteration is:
[263]	training's auc: 0.886458	training's binary_logloss: 0.267826	valid_1's auc: 0.854597	valid_1's binary_logloss: 0.293717
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852371	training's b

In [77]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 836
variables con importancia acumulada al 99% : 810
variables con zero importancia : 0


In [78]:
keep_columns_v13 = keep_columns

In [79]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852495	training's binary_logloss: 0.296307	valid_1's auc: 0.843528	valid_1's binary_logloss: 0.302985
[100]	training's auc: 0.86515	training's binary_logloss: 0.285211	valid_1's auc: 0.850721	valid_1's binary_logloss: 0.296789
[150]	training's auc: 0.873016	training's binary_logloss: 0.27873	valid_1's auc: 0.853013	valid_1's binary_logloss: 0.294861
[200]	training's auc: 0.879316	training's binary_logloss: 0.273632	valid_1's auc: 0.853924	valid_1's binary_logloss: 0.294134
Early stopping, best iteration is:
[219]	training's auc: 0.881481	training's binary_logloss: 0.271884	valid_1's auc: 0.854336	valid_1's binary_logloss: 0.293875
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85208	training's binary_logloss: 0.296415	valid_1's auc: 0.843778	valid_1's binary_logloss: 0.302842
[100]	training's auc: 0.865026	training's bi

In [80]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 810
variables con importancia acumulada al 99% : 784
variables con zero importancia : 0


In [81]:
keep_columns_v14 = keep_columns

In [82]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852209	training's binary_logloss: 0.296339	valid_1's auc: 0.843274	valid_1's binary_logloss: 0.302992
[100]	training's auc: 0.864976	training's binary_logloss: 0.285315	valid_1's auc: 0.851218	valid_1's binary_logloss: 0.296533
[150]	training's auc: 0.872673	training's binary_logloss: 0.278866	valid_1's auc: 0.853562	valid_1's binary_logloss: 0.294612
[200]	training's auc: 0.879054	training's binary_logloss: 0.273741	valid_1's auc: 0.854508	valid_1's binary_logloss: 0.293846
[250]	training's auc: 0.884683	training's binary_logloss: 0.269167	valid_1's auc: 0.855069	valid_1's binary_logloss: 0.293426
Early stopping, best iteration is:
[242]	training's auc: 0.883778	training's binary_logloss: 0.269884	valid_1's auc: 0.855135	valid_1's binary_logloss: 0.29338
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852174	training's 

In [83]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 784
variables con importancia acumulada al 99% : 761
variables con zero importancia : 0


In [84]:
keep_columns_v15 = keep_columns

In [85]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852436	training's binary_logloss: 0.296252	valid_1's auc: 0.843297	valid_1's binary_logloss: 0.303046
[100]	training's auc: 0.865049	training's binary_logloss: 0.285197	valid_1's auc: 0.8508	valid_1's binary_logloss: 0.296703
[150]	training's auc: 0.872947	training's binary_logloss: 0.278809	valid_1's auc: 0.853304	valid_1's binary_logloss: 0.294743
[200]	training's auc: 0.879232	training's binary_logloss: 0.273675	valid_1's auc: 0.854285	valid_1's binary_logloss: 0.293859
[250]	training's auc: 0.884921	training's binary_logloss: 0.269126	valid_1's auc: 0.854885	valid_1's binary_logloss: 0.293467
Early stopping, best iteration is:
[273]	training's auc: 0.887172	training's binary_logloss: 0.267174	valid_1's auc: 0.855028	valid_1's binary_logloss: 0.29334
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852297	training's bi

In [86]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 761
variables con importancia acumulada al 99% : 740
variables con zero importancia : 0


In [87]:
keep_columns_v16 = keep_columns

In [88]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852459	training's binary_logloss: 0.296232	valid_1's auc: 0.843486	valid_1's binary_logloss: 0.302772
[100]	training's auc: 0.864891	training's binary_logloss: 0.285336	valid_1's auc: 0.850996	valid_1's binary_logloss: 0.296538
[150]	training's auc: 0.872772	training's binary_logloss: 0.278911	valid_1's auc: 0.853248	valid_1's binary_logloss: 0.294721
[200]	training's auc: 0.879191	training's binary_logloss: 0.27375	valid_1's auc: 0.854096	valid_1's binary_logloss: 0.29402
[250]	training's auc: 0.884959	training's binary_logloss: 0.269095	valid_1's auc: 0.854514	valid_1's binary_logloss: 0.29362
Early stopping, best iteration is:
[249]	training's auc: 0.884846	training's binary_logloss: 0.269184	valid_1's auc: 0.854555	valid_1's binary_logloss: 0.293599
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852554	training's bi

In [89]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 740
variables con importancia acumulada al 99% : 718
variables con zero importancia : 0


In [90]:
keep_columns_v17 = keep_columns

In [91]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852534	training's binary_logloss: 0.296244	valid_1's auc: 0.843267	valid_1's binary_logloss: 0.303074
[100]	training's auc: 0.864944	training's binary_logloss: 0.285314	valid_1's auc: 0.850627	valid_1's binary_logloss: 0.296959
[150]	training's auc: 0.872867	training's binary_logloss: 0.278882	valid_1's auc: 0.852918	valid_1's binary_logloss: 0.295083
[200]	training's auc: 0.879226	training's binary_logloss: 0.273722	valid_1's auc: 0.854088	valid_1's binary_logloss: 0.294165
[250]	training's auc: 0.884852	training's binary_logloss: 0.269179	valid_1's auc: 0.854818	valid_1's binary_logloss: 0.293647
Early stopping, best iteration is:
[252]	training's auc: 0.885093	training's binary_logloss: 0.269005	valid_1's auc: 0.854877	valid_1's binary_logloss: 0.293591
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852053	training's

In [92]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 718
variables con importancia acumulada al 99% : 699
variables con zero importancia : 0


In [93]:
keep_columns_v18 = keep_columns

In [94]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85225	training's binary_logloss: 0.296336	valid_1's auc: 0.843297	valid_1's binary_logloss: 0.303101
[100]	training's auc: 0.865063	training's binary_logloss: 0.285307	valid_1's auc: 0.851073	valid_1's binary_logloss: 0.296803
[150]	training's auc: 0.872816	training's binary_logloss: 0.278824	valid_1's auc: 0.853287	valid_1's binary_logloss: 0.294872
[200]	training's auc: 0.879366	training's binary_logloss: 0.27362	valid_1's auc: 0.854257	valid_1's binary_logloss: 0.294067
[250]	training's auc: 0.884813	training's binary_logloss: 0.269178	valid_1's auc: 0.854639	valid_1's binary_logloss: 0.293767
Early stopping, best iteration is:
[258]	training's auc: 0.885667	training's binary_logloss: 0.268462	valid_1's auc: 0.854674	valid_1's binary_logloss: 0.293727
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852184	training's b

In [95]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 699
variables con importancia acumulada al 99% : 680
variables con zero importancia : 0


In [96]:
keep_columns_v19 = keep_columns

In [97]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852396	training's binary_logloss: 0.296242	valid_1's auc: 0.843327	valid_1's binary_logloss: 0.302942
[100]	training's auc: 0.86488	training's binary_logloss: 0.285336	valid_1's auc: 0.850616	valid_1's binary_logloss: 0.296876
[150]	training's auc: 0.872806	training's binary_logloss: 0.278903	valid_1's auc: 0.853093	valid_1's binary_logloss: 0.294848
[200]	training's auc: 0.879241	training's binary_logloss: 0.273759	valid_1's auc: 0.853946	valid_1's binary_logloss: 0.294102
[250]	training's auc: 0.8848	training's binary_logloss: 0.26919	valid_1's auc: 0.854604	valid_1's binary_logloss: 0.293589
[300]	training's auc: 0.889931	training's binary_logloss: 0.264967	valid_1's auc: 0.854985	valid_1's binary_logloss: 0.293313
Early stopping, best iteration is:
[293]	training's auc: 0.889199	training's binary_logloss: 0.265553	valid_1's auc: 0.855023	valid_1's binary_logloss: 0.293297
***

In [98]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 680
variables con importancia acumulada al 99% : 662
variables con zero importancia : 0


In [99]:
keep_columns_v20 = keep_columns

In [100]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852173	training's binary_logloss: 0.296362	valid_1's auc: 0.843336	valid_1's binary_logloss: 0.303077
[100]	training's auc: 0.864872	training's binary_logloss: 0.285383	valid_1's auc: 0.850833	valid_1's binary_logloss: 0.296829
[150]	training's auc: 0.872657	training's binary_logloss: 0.278982	valid_1's auc: 0.853023	valid_1's binary_logloss: 0.295018
[200]	training's auc: 0.878913	training's binary_logloss: 0.273893	valid_1's auc: 0.853784	valid_1's binary_logloss: 0.294285
Early stopping, best iteration is:
[196]	training's auc: 0.878485	training's binary_logloss: 0.274254	valid_1's auc: 0.853793	valid_1's binary_logloss: 0.294302
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852351	training's binary_logloss: 0.296352	valid_1's auc: 0.844038	valid_1's binary_logloss: 0.302649
[100]	training's auc: 0.865086	training's

In [101]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 662
variables con importancia acumulada al 99% : 645
variables con zero importancia : 0


In [102]:
keep_columns_v21 = keep_columns

In [103]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852173	training's binary_logloss: 0.296387	valid_1's auc: 0.842979	valid_1's binary_logloss: 0.303156
[100]	training's auc: 0.864977	training's binary_logloss: 0.285363	valid_1's auc: 0.850453	valid_1's binary_logloss: 0.296973
[150]	training's auc: 0.872672	training's binary_logloss: 0.278924	valid_1's auc: 0.852512	valid_1's binary_logloss: 0.29516
[200]	training's auc: 0.878963	training's binary_logloss: 0.273786	valid_1's auc: 0.853322	valid_1's binary_logloss: 0.294467
Early stopping, best iteration is:
[200]	training's auc: 0.878963	training's binary_logloss: 0.273786	valid_1's auc: 0.853322	valid_1's binary_logloss: 0.294467
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852271	training's binary_logloss: 0.296162	valid_1's auc: 0.844037	valid_1's binary_logloss: 0.302531
[100]	training's auc: 0.865101	training's 

In [104]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 645
variables con importancia acumulada al 99% : 627
variables con zero importancia : 0


In [105]:
keep_columns_v22 = keep_columns

In [106]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.8519	training's binary_logloss: 0.296416	valid_1's auc: 0.843723	valid_1's binary_logloss: 0.302862
[100]	training's auc: 0.864673	training's binary_logloss: 0.285408	valid_1's auc: 0.85124	valid_1's binary_logloss: 0.296563
[150]	training's auc: 0.872491	training's binary_logloss: 0.279024	valid_1's auc: 0.853743	valid_1's binary_logloss: 0.294559
[200]	training's auc: 0.878773	training's binary_logloss: 0.27396	valid_1's auc: 0.854514	valid_1's binary_logloss: 0.29381
[250]	training's auc: 0.88443	training's binary_logloss: 0.26935	valid_1's auc: 0.855221	valid_1's binary_logloss: 0.293327
Early stopping, best iteration is:
[260]	training's auc: 0.885449	training's binary_logloss: 0.268475	valid_1's auc: 0.855249	valid_1's binary_logloss: 0.293319
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852139	training's binary

In [107]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 627
variables con importancia acumulada al 99% : 610
variables con zero importancia : 0


In [108]:
keep_columns_v23 = keep_columns

In [109]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852391	training's binary_logloss: 0.29627	valid_1's auc: 0.843754	valid_1's binary_logloss: 0.302814
[100]	training's auc: 0.86502	training's binary_logloss: 0.285229	valid_1's auc: 0.851056	valid_1's binary_logloss: 0.296668
[150]	training's auc: 0.872872	training's binary_logloss: 0.278855	valid_1's auc: 0.853305	valid_1's binary_logloss: 0.294764
[200]	training's auc: 0.87912	training's binary_logloss: 0.273815	valid_1's auc: 0.854326	valid_1's binary_logloss: 0.293945
Early stopping, best iteration is:
[194]	training's auc: 0.878385	training's binary_logloss: 0.274382	valid_1's auc: 0.854356	valid_1's binary_logloss: 0.293933
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852108	training's binary_logloss: 0.296367	valid_1's auc: 0.843991	valid_1's binary_logloss: 0.302774
[100]	training's auc: 0.864924	training's bi

In [110]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 610
variables con importancia acumulada al 99% : 594
variables con zero importancia : 0


In [111]:
keep_columns_v24 = keep_columns

In [112]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852212	training's binary_logloss: 0.296347	valid_1's auc: 0.843445	valid_1's binary_logloss: 0.302907
[100]	training's auc: 0.864867	training's binary_logloss: 0.285371	valid_1's auc: 0.850742	valid_1's binary_logloss: 0.296836
[150]	training's auc: 0.872572	training's binary_logloss: 0.279042	valid_1's auc: 0.852914	valid_1's binary_logloss: 0.295029
[200]	training's auc: 0.878825	training's binary_logloss: 0.273939	valid_1's auc: 0.85391	valid_1's binary_logloss: 0.294256
[250]	training's auc: 0.88454	training's binary_logloss: 0.269393	valid_1's auc: 0.854597	valid_1's binary_logloss: 0.293759
Early stopping, best iteration is:
[255]	training's auc: 0.885033	training's binary_logloss: 0.268968	valid_1's auc: 0.854671	valid_1's binary_logloss: 0.29371
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852216	training's bi

In [113]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 594
variables con importancia acumulada al 99% : 579
variables con zero importancia : 0


In [114]:
keep_columns_v25 = keep_columns

In [115]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852535	training's binary_logloss: 0.296277	valid_1's auc: 0.84366	valid_1's binary_logloss: 0.302988
[100]	training's auc: 0.865026	training's binary_logloss: 0.285317	valid_1's auc: 0.851069	valid_1's binary_logloss: 0.296814
[150]	training's auc: 0.872734	training's binary_logloss: 0.278921	valid_1's auc: 0.852978	valid_1's binary_logloss: 0.295059
[200]	training's auc: 0.879065	training's binary_logloss: 0.273797	valid_1's auc: 0.853861	valid_1's binary_logloss: 0.294272
[250]	training's auc: 0.884581	training's binary_logloss: 0.269286	valid_1's auc: 0.854469	valid_1's binary_logloss: 0.293821
Early stopping, best iteration is:
[261]	training's auc: 0.885789	training's binary_logloss: 0.268329	valid_1's auc: 0.854555	valid_1's binary_logloss: 0.293789
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851925	training's 

In [116]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 579
variables con importancia acumulada al 99% : 564
variables con zero importancia : 0


In [117]:
keep_columns_v26 = keep_columns

In [118]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852376	training's binary_logloss: 0.296294	valid_1's auc: 0.843722	valid_1's binary_logloss: 0.30285
[100]	training's auc: 0.864971	training's binary_logloss: 0.285357	valid_1's auc: 0.851312	valid_1's binary_logloss: 0.296543
[150]	training's auc: 0.872737	training's binary_logloss: 0.27897	valid_1's auc: 0.853446	valid_1's binary_logloss: 0.294749
[200]	training's auc: 0.879097	training's binary_logloss: 0.273915	valid_1's auc: 0.854241	valid_1's binary_logloss: 0.294052
Early stopping, best iteration is:
[222]	training's auc: 0.881537	training's binary_logloss: 0.271868	valid_1's auc: 0.854611	valid_1's binary_logloss: 0.293761
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851819	training's binary_logloss: 0.296593	valid_1's auc: 0.8435	valid_1's binary_logloss: 0.302975
[100]	training's auc: 0.864932	training's bin

### elegimos la mejor combinacion de variables que es 680

In [119]:
len(keep_columns_v19)

680

In [120]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns_v19], y_train, test[keep_columns_v19])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852396	training's binary_logloss: 0.296242	valid_1's auc: 0.843327	valid_1's binary_logloss: 0.302942
[100]	training's auc: 0.86488	training's binary_logloss: 0.285336	valid_1's auc: 0.850616	valid_1's binary_logloss: 0.296876
[150]	training's auc: 0.872806	training's binary_logloss: 0.278903	valid_1's auc: 0.853093	valid_1's binary_logloss: 0.294848
[200]	training's auc: 0.879241	training's binary_logloss: 0.273759	valid_1's auc: 0.853946	valid_1's binary_logloss: 0.294102
[250]	training's auc: 0.8848	training's binary_logloss: 0.26919	valid_1's auc: 0.854604	valid_1's binary_logloss: 0.293589
[300]	training's auc: 0.889931	training's binary_logloss: 0.264967	valid_1's auc: 0.854985	valid_1's binary_logloss: 0.293313
Early stopping, best iteration is:
[293]	training's auc: 0.889199	training's binary_logloss: 0.265553	valid_1's auc: 0.855023	valid_1's binary_logloss: 0.293297
***

In [121]:
test_probs.name='target'
test_probs.to_csv('../results/lightgbm_with_680features_score_0.85386.csv') ### score de 0.85558 en la tabla publica

In [128]:
keep_columns_list = [keep_columns_v2, keep_columns_v4, keep_columns_v5, keep_columns_v6, keep_columns_v7,
keep_columns_v8, keep_columns_v9, keep_columns_v10, keep_columns_v11, keep_columns_v12, keep_columns_v13,
keep_columns_v14, keep_columns_v15, keep_columns_v16, keep_columns_v17, keep_columns_v18, keep_columns_v19,
keep_columns_v20, keep_columns_v21, keep_columns_v22, keep_columns_v23, keep_columns_v24, keep_columns_v25,
keep_columns_v26]
np.save('keep_columns_list.npy', np.array(keep_columns_list))

<ipython-input-128-f1f198724c3d>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.save('keep_columns_list.npy', np.array(keep_columns_list))


In [132]:
train = train[keep_columns_v19]
test = test[keep_columns_v19]

In [134]:
train.shape, test.shape

((358487, 680), (396666, 680))

In [135]:
# train.to_pickle('../data/train_features680_score_0.85558.pkl')
# test.to_pickle('../data/test_features680_score_0.85558.pkl')

In [133]:
aux = (train.isna().sum()/len(train)).sort_values()
features_without_na = aux[aux==0]
print(f'feature sin missing values {len(features_without_na)} representando el {round(len(features_without_na)/len(train.columns), 3)} de features totales')
list_prec_na = [(0,0.1), (0.1,0.2), (0.2,0.3), (0.3,0.4), (0.4,0.5), (0.5,0.6), (0.6,0.7), (0.7,0.8), (0.8,0.9),(0.9,1.1)]
for i in list_prec_na:
    features_with_na = aux[(aux>=i[0]) & (aux<i[1])]
    print(f'features con porc de missing values de {i[0]} a {i[1]} : {len(features_with_na)} representando el {round(len(features_with_na)/len(train.columns), 3)} de features totales')

feature sin missing values 68 representando el 0.1 de features totales
features con porc de missing values de 0 a 0.1 : 171 representando el 0.251 de features totales
features con porc de missing values de 0.1 a 0.2 : 8 representando el 0.012 de features totales
features con porc de missing values de 0.2 a 0.3 : 1 representando el 0.001 de features totales
features con porc de missing values de 0.3 a 0.4 : 23 representando el 0.034 de features totales
features con porc de missing values de 0.4 a 0.5 : 103 representando el 0.151 de features totales
features con porc de missing values de 0.5 a 0.6 : 58 representando el 0.085 de features totales
features con porc de missing values de 0.6 a 0.7 : 19 representando el 0.028 de features totales
features con porc de missing values de 0.7 a 0.8 : 44 representando el 0.065 de features totales
features con porc de missing values de 0.8 a 0.9 : 96 representando el 0.141 de features totales
features con porc de missing values de 0.9 a 1.1 : 157 rep

### Continuando con la reduccion de variables

In [11]:
train = pd.read_pickle('../data/train_features680_score_0.85558.pkl')
test = pd.read_pickle('../data/test_features680_score_0.85558.pkl')

In [12]:
import numpy as np
keep_columns_list = list(np.load('keep_columns_list.npy', allow_pickle=True))

In [6]:
len(keep_columns_list[-1])

564

In [13]:
keep_columns = keep_columns_list[-1]

In [14]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852376	training's binary_logloss: 0.296294	valid_1's auc: 0.843722	valid_1's binary_logloss: 0.30285
[100]	training's auc: 0.864971	training's binary_logloss: 0.285357	valid_1's auc: 0.851312	valid_1's binary_logloss: 0.296543
[150]	training's auc: 0.872737	training's binary_logloss: 0.27897	valid_1's auc: 0.853446	valid_1's binary_logloss: 0.294749
[200]	training's auc: 0.879097	training's binary_logloss: 0.273915	valid_1's auc: 0.854241	valid_1's binary_logloss: 0.294052
Early stopping, best iteration is:
[222]	training's auc: 0.881537	training's binary_logloss: 0.271868	valid_1's auc: 0.854611	valid_1's binary_logloss: 0.293761
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851819	training's binary_logloss: 0.296593	valid_1's auc: 0.8435	valid_1's binary_logloss: 0.302975
[100]	training's auc: 0.864932	training's bin

In [15]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 564
variables con importancia acumulada al 99% : 551
variables con zero importancia : 0


In [16]:
keep_columns_v27 = keep_columns

In [17]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852351	training's binary_logloss: 0.296361	valid_1's auc: 0.843354	valid_1's binary_logloss: 0.303014
[100]	training's auc: 0.864996	training's binary_logloss: 0.285391	valid_1's auc: 0.850832	valid_1's binary_logloss: 0.296821
[150]	training's auc: 0.872745	training's binary_logloss: 0.279039	valid_1's auc: 0.852868	valid_1's binary_logloss: 0.295108
Early stopping, best iteration is:
[188]	training's auc: 0.877445	training's binary_logloss: 0.275177	valid_1's auc: 0.853664	valid_1's binary_logloss: 0.29445
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85247	training's binary_logloss: 0.296238	valid_1's auc: 0.844365	valid_1's binary_logloss: 0.302404
[100]	training's auc: 0.865094	training's binary_logloss: 0.285401	valid_1's auc: 0.851237	valid_1's binary_logloss: 0.296397
[150]	training's auc: 0.872691	training's b

In [18]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 551
variables con importancia acumulada al 99% : 538
variables con zero importancia : 0


In [19]:
keep_columns_v28 = keep_columns

In [20]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852243	training's binary_logloss: 0.296377	valid_1's auc: 0.843327	valid_1's binary_logloss: 0.302992
[100]	training's auc: 0.8649	training's binary_logloss: 0.285367	valid_1's auc: 0.850938	valid_1's binary_logloss: 0.296734
[150]	training's auc: 0.872737	training's binary_logloss: 0.279009	valid_1's auc: 0.853442	valid_1's binary_logloss: 0.294783
[200]	training's auc: 0.878916	training's binary_logloss: 0.273986	valid_1's auc: 0.854195	valid_1's binary_logloss: 0.294148
[250]	training's auc: 0.884613	training's binary_logloss: 0.269425	valid_1's auc: 0.854624	valid_1's binary_logloss: 0.293848
Early stopping, best iteration is:
[276]	training's auc: 0.887253	training's binary_logloss: 0.267213	valid_1's auc: 0.854815	valid_1's binary_logloss: 0.293658
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852403	training's b

In [21]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 538
variables con importancia acumulada al 99% : 525
variables con zero importancia : 0


In [22]:
keep_columns_v29 = keep_columns

In [23]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852335	training's binary_logloss: 0.296445	valid_1's auc: 0.843197	valid_1's binary_logloss: 0.303147
[100]	training's auc: 0.864892	training's binary_logloss: 0.285432	valid_1's auc: 0.850853	valid_1's binary_logloss: 0.296788
[150]	training's auc: 0.872787	training's binary_logloss: 0.279066	valid_1's auc: 0.853087	valid_1's binary_logloss: 0.295026
[200]	training's auc: 0.879007	training's binary_logloss: 0.273987	valid_1's auc: 0.853864	valid_1's binary_logloss: 0.294324
[250]	training's auc: 0.88452	training's binary_logloss: 0.269532	valid_1's auc: 0.854341	valid_1's binary_logloss: 0.294028
Early stopping, best iteration is:
[262]	training's auc: 0.885758	training's binary_logloss: 0.268538	valid_1's auc: 0.854463	valid_1's binary_logloss: 0.293932
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851883	training's 

In [24]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 525
variables con importancia acumulada al 99% : 513
variables con zero importancia : 0


In [25]:
keep_columns_v30 = keep_columns

In [26]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852408	training's binary_logloss: 0.296248	valid_1's auc: 0.843762	valid_1's binary_logloss: 0.302859
[100]	training's auc: 0.864966	training's binary_logloss: 0.285348	valid_1's auc: 0.851114	valid_1's binary_logloss: 0.296757
[150]	training's auc: 0.872684	training's binary_logloss: 0.278992	valid_1's auc: 0.853305	valid_1's binary_logloss: 0.29487
[200]	training's auc: 0.878874	training's binary_logloss: 0.273995	valid_1's auc: 0.854171	valid_1's binary_logloss: 0.294161
Early stopping, best iteration is:
[199]	training's auc: 0.878762	training's binary_logloss: 0.274084	valid_1's auc: 0.85421	valid_1's binary_logloss: 0.294141
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85182	training's binary_logloss: 0.296635	valid_1's auc: 0.843308	valid_1's binary_logloss: 0.303052
[100]	training's auc: 0.864651	training's bi

In [27]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 513
variables con importancia acumulada al 99% : 501
variables con zero importancia : 0


In [28]:
keep_columns_v31 = keep_columns

In [29]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85212	training's binary_logloss: 0.296487	valid_1's auc: 0.843427	valid_1's binary_logloss: 0.303171
[100]	training's auc: 0.864716	training's binary_logloss: 0.285503	valid_1's auc: 0.850749	valid_1's binary_logloss: 0.296966
[150]	training's auc: 0.872612	training's binary_logloss: 0.279096	valid_1's auc: 0.853132	valid_1's binary_logloss: 0.295032
[200]	training's auc: 0.878876	training's binary_logloss: 0.274001	valid_1's auc: 0.853842	valid_1's binary_logloss: 0.294393
Early stopping, best iteration is:
[237]	training's auc: 0.883034	training's binary_logloss: 0.27065	valid_1's auc: 0.854348	valid_1's binary_logloss: 0.294009
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852119	training's binary_logloss: 0.296397	valid_1's auc: 0.843293	valid_1's binary_logloss: 0.302948
[100]	training's auc: 0.86501	training's bi

In [30]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 501
variables con importancia acumulada al 99% : 489
variables con zero importancia : 0


In [31]:
keep_columns_v32 = keep_columns

In [32]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85226	training's binary_logloss: 0.296502	valid_1's auc: 0.843597	valid_1's binary_logloss: 0.302991
[100]	training's auc: 0.864594	training's binary_logloss: 0.285677	valid_1's auc: 0.850831	valid_1's binary_logloss: 0.296908
[150]	training's auc: 0.872446	training's binary_logloss: 0.27928	valid_1's auc: 0.853053	valid_1's binary_logloss: 0.295
[200]	training's auc: 0.878845	training's binary_logloss: 0.274167	valid_1's auc: 0.854139	valid_1's binary_logloss: 0.294137
[250]	training's auc: 0.884202	training's binary_logloss: 0.269737	valid_1's auc: 0.854652	valid_1's binary_logloss: 0.293716
[300]	training's auc: 0.889429	training's binary_logloss: 0.265511	valid_1's auc: 0.855156	valid_1's binary_logloss: 0.293362
Early stopping, best iteration is:
[299]	training's auc: 0.88933	training's binary_logloss: 0.2656	valid_1's auc: 0.855165	valid_1's binary_logloss: 0.29335
********

In [36]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 489
variables con importancia acumulada al 99% : 478
variables con zero importancia : 0


In [37]:
keep_columns_v33 = keep_columns

In [38]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852039	training's binary_logloss: 0.296505	valid_1's auc: 0.843018	valid_1's binary_logloss: 0.303322
[100]	training's auc: 0.86475	training's binary_logloss: 0.285623	valid_1's auc: 0.850604	valid_1's binary_logloss: 0.297274
[150]	training's auc: 0.87243	training's binary_logloss: 0.279334	valid_1's auc: 0.852848	valid_1's binary_logloss: 0.295454
[200]	training's auc: 0.87863	training's binary_logloss: 0.274274	valid_1's auc: 0.853781	valid_1's binary_logloss: 0.294744
Early stopping, best iteration is:
[235]	training's auc: 0.882405	training's binary_logloss: 0.271138	valid_1's auc: 0.854135	valid_1's binary_logloss: 0.294443
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852006	training's binary_logloss: 0.296448	valid_1's auc: 0.844155	valid_1's binary_logloss: 0.302612
[100]	training's auc: 0.864672	training's bi

In [39]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 478
variables con importancia acumulada al 99% : 467
variables con zero importancia : 0


In [40]:
keep_columns_v34 = keep_columns

In [41]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85204	training's binary_logloss: 0.296496	valid_1's auc: 0.842684	valid_1's binary_logloss: 0.303361
[100]	training's auc: 0.864977	training's binary_logloss: 0.285558	valid_1's auc: 0.850629	valid_1's binary_logloss: 0.297157
[150]	training's auc: 0.872634	training's binary_logloss: 0.279233	valid_1's auc: 0.852911	valid_1's binary_logloss: 0.295227
[200]	training's auc: 0.878926	training's binary_logloss: 0.27413	valid_1's auc: 0.853775	valid_1's binary_logloss: 0.294498
Early stopping, best iteration is:
[231]	training's auc: 0.882463	training's binary_logloss: 0.27124	valid_1's auc: 0.85402	valid_1's binary_logloss: 0.294316
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852132	training's binary_logloss: 0.296434	valid_1's auc: 0.843704	valid_1's binary_logloss: 0.3027
[100]	training's auc: 0.86484	training's binary

In [42]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 467
variables con importancia acumulada al 99% : 456
variables con zero importancia : 0


In [43]:
keep_columns_v35 = keep_columns

In [44]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852127	training's binary_logloss: 0.296523	valid_1's auc: 0.842965	valid_1's binary_logloss: 0.303334
[100]	training's auc: 0.864775	training's binary_logloss: 0.285588	valid_1's auc: 0.850368	valid_1's binary_logloss: 0.297222
[150]	training's auc: 0.872378	training's binary_logloss: 0.279293	valid_1's auc: 0.852519	valid_1's binary_logloss: 0.295383
[200]	training's auc: 0.878611	training's binary_logloss: 0.27427	valid_1's auc: 0.853516	valid_1's binary_logloss: 0.294621
[250]	training's auc: 0.88412	training's binary_logloss: 0.269826	valid_1's auc: 0.854	valid_1's binary_logloss: 0.29422
[300]	training's auc: 0.889144	training's binary_logloss: 0.265695	valid_1's auc: 0.854351	valid_1's binary_logloss: 0.293954
Early stopping, best iteration is:
[303]	training's auc: 0.88946	training's binary_logloss: 0.26544	valid_1's auc: 0.854374	valid_1's binary_logloss: 0.293939
*******

In [45]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 456
variables con importancia acumulada al 99% : 446
variables con zero importancia : 0


In [46]:
keep_columns_v36 = keep_columns

In [47]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852174	training's binary_logloss: 0.296512	valid_1's auc: 0.843288	valid_1's binary_logloss: 0.303265
[100]	training's auc: 0.86475	training's binary_logloss: 0.285608	valid_1's auc: 0.85052	valid_1's binary_logloss: 0.297134
[150]	training's auc: 0.872556	training's binary_logloss: 0.279247	valid_1's auc: 0.852648	valid_1's binary_logloss: 0.295422
[200]	training's auc: 0.878714	training's binary_logloss: 0.274285	valid_1's auc: 0.85388	valid_1's binary_logloss: 0.294479
Early stopping, best iteration is:
[220]	training's auc: 0.880966	training's binary_logloss: 0.272442	valid_1's auc: 0.854263	valid_1's binary_logloss: 0.294151
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852035	training's binary_logloss: 0.296623	valid_1's auc: 0.844094	valid_1's binary_logloss: 0.302775
[100]	training's auc: 0.864676	training's bi

In [48]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 446
variables con importancia acumulada al 99% : 436
variables con zero importancia : 0


In [49]:
keep_columns_v37 = keep_columns

In [50]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852155	training's binary_logloss: 0.296587	valid_1's auc: 0.843199	valid_1's binary_logloss: 0.303299
[100]	training's auc: 0.864718	training's binary_logloss: 0.285683	valid_1's auc: 0.850722	valid_1's binary_logloss: 0.297097
[150]	training's auc: 0.872231	training's binary_logloss: 0.279426	valid_1's auc: 0.852717	valid_1's binary_logloss: 0.295291
[200]	training's auc: 0.878518	training's binary_logloss: 0.274435	valid_1's auc: 0.853614	valid_1's binary_logloss: 0.294552
Early stopping, best iteration is:
[238]	training's auc: 0.882774	training's binary_logloss: 0.270964	valid_1's auc: 0.853998	valid_1's binary_logloss: 0.29426
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852044	training's binary_logloss: 0.296433	valid_1's auc: 0.843532	valid_1's binary_logloss: 0.302859
[100]	training's auc: 0.864653	training's 

In [51]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 436
variables con importancia acumulada al 99% : 425
variables con zero importancia : 0


In [52]:
keep_columns_v38 = keep_columns

In [53]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851901	training's binary_logloss: 0.296688	valid_1's auc: 0.843041	valid_1's binary_logloss: 0.303307
[100]	training's auc: 0.864393	training's binary_logloss: 0.285864	valid_1's auc: 0.85085	valid_1's binary_logloss: 0.297022
[150]	training's auc: 0.872057	training's binary_logloss: 0.279588	valid_1's auc: 0.852909	valid_1's binary_logloss: 0.295286
[200]	training's auc: 0.878477	training's binary_logloss: 0.274451	valid_1's auc: 0.85362	valid_1's binary_logloss: 0.294695
Early stopping, best iteration is:
[228]	training's auc: 0.881647	training's binary_logloss: 0.271906	valid_1's auc: 0.853982	valid_1's binary_logloss: 0.29446
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851948	training's binary_logloss: 0.296672	valid_1's auc: 0.843397	valid_1's binary_logloss: 0.303271
[100]	training's auc: 0.864449	training's bi

In [54]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 425
variables con importancia acumulada al 99% : 415
variables con zero importancia : 0


In [55]:
keep_columns_v39 = keep_columns

In [56]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852036	training's binary_logloss: 0.296527	valid_1's auc: 0.843216	valid_1's binary_logloss: 0.303153
[100]	training's auc: 0.864527	training's binary_logloss: 0.285813	valid_1's auc: 0.850582	valid_1's binary_logloss: 0.297146
[150]	training's auc: 0.87222	training's binary_logloss: 0.279469	valid_1's auc: 0.852816	valid_1's binary_logloss: 0.29528
[200]	training's auc: 0.878394	training's binary_logloss: 0.27452	valid_1's auc: 0.853759	valid_1's binary_logloss: 0.294518
[250]	training's auc: 0.883674	training's binary_logloss: 0.270047	valid_1's auc: 0.854292	valid_1's binary_logloss: 0.29405
Early stopping, best iteration is:
[248]	training's auc: 0.883468	training's binary_logloss: 0.270213	valid_1's auc: 0.854315	valid_1's binary_logloss: 0.294045
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851942	training's bin

In [57]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 415
variables con importancia acumulada al 99% : 406
variables con zero importancia : 0


In [58]:
keep_columns_v40 = keep_columns

In [59]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851808	training's binary_logloss: 0.296684	valid_1's auc: 0.843328	valid_1's binary_logloss: 0.303263
[100]	training's auc: 0.864198	training's binary_logloss: 0.285961	valid_1's auc: 0.850433	valid_1's binary_logloss: 0.297282
[150]	training's auc: 0.871966	training's binary_logloss: 0.279653	valid_1's auc: 0.852838	valid_1's binary_logloss: 0.295413
Early stopping, best iteration is:
[184]	training's auc: 0.876072	training's binary_logloss: 0.276213	valid_1's auc: 0.853529	valid_1's binary_logloss: 0.294813
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851921	training's binary_logloss: 0.296774	valid_1's auc: 0.843595	valid_1's binary_logloss: 0.302979
[100]	training's auc: 0.864622	training's binary_logloss: 0.285786	valid_1's auc: 0.850926	valid_1's binary_logloss: 0.296608
[150]	training's auc: 0.87225	training's 

In [60]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 406
variables con importancia acumulada al 99% : 397
variables con zero importancia : 0


In [61]:
keep_columns_v41 = keep_columns

In [62]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85201	training's binary_logloss: 0.29667	valid_1's auc: 0.843466	valid_1's binary_logloss: 0.303306
[100]	training's auc: 0.864574	training's binary_logloss: 0.285876	valid_1's auc: 0.851106	valid_1's binary_logloss: 0.297129
[150]	training's auc: 0.872169	training's binary_logloss: 0.279504	valid_1's auc: 0.853527	valid_1's binary_logloss: 0.294962
Early stopping, best iteration is:
[178]	training's auc: 0.875734	training's binary_logloss: 0.27664	valid_1's auc: 0.854082	valid_1's binary_logloss: 0.294555
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85159	training's binary_logloss: 0.296848	valid_1's auc: 0.843391	valid_1's binary_logloss: 0.303083
[100]	training's auc: 0.864355	training's binary_logloss: 0.285854	valid_1's auc: 0.850801	valid_1's binary_logloss: 0.296643
[150]	training's auc: 0.871927	training's bin

In [63]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 397
variables con importancia acumulada al 99% : 388
variables con zero importancia : 0


In [64]:
keep_columns_v42 = keep_columns

In [65]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851637	training's binary_logloss: 0.296807	valid_1's auc: 0.843236	valid_1's binary_logloss: 0.303408
[100]	training's auc: 0.864402	training's binary_logloss: 0.285949	valid_1's auc: 0.850404	valid_1's binary_logloss: 0.297494
[150]	training's auc: 0.872071	training's binary_logloss: 0.279652	valid_1's auc: 0.85267	valid_1's binary_logloss: 0.295604
[200]	training's auc: 0.878337	training's binary_logloss: 0.274612	valid_1's auc: 0.853543	valid_1's binary_logloss: 0.294899
[250]	training's auc: 0.8838	training's binary_logloss: 0.270138	valid_1's auc: 0.854074	valid_1's binary_logloss: 0.294441
Early stopping, best iteration is:
[246]	training's auc: 0.883329	training's binary_logloss: 0.270486	valid_1's auc: 0.854107	valid_1's binary_logloss: 0.294422
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851991	training's bi

In [66]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 388
variables con importancia acumulada al 99% : 380
variables con zero importancia : 0


In [67]:
keep_columns_v43 = keep_columns

In [68]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851699	training's binary_logloss: 0.296834	valid_1's auc: 0.843057	valid_1's binary_logloss: 0.303481
[100]	training's auc: 0.864267	training's binary_logloss: 0.285966	valid_1's auc: 0.850581	valid_1's binary_logloss: 0.297328
[150]	training's auc: 0.871911	training's binary_logloss: 0.279686	valid_1's auc: 0.852653	valid_1's binary_logloss: 0.29555
[200]	training's auc: 0.878044	training's binary_logloss: 0.274715	valid_1's auc: 0.853376	valid_1's binary_logloss: 0.294912
Early stopping, best iteration is:
[220]	training's auc: 0.880305	training's binary_logloss: 0.272914	valid_1's auc: 0.853599	valid_1's binary_logloss: 0.294713
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851746	training's binary_logloss: 0.296763	valid_1's auc: 0.843869	valid_1's binary_logloss: 0.302968
[100]	training's auc: 0.864329	training's 

In [69]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 380
variables con importancia acumulada al 99% : 372
variables con zero importancia : 0


In [70]:
keep_columns_v44 = keep_columns

In [71]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851912	training's binary_logloss: 0.296731	valid_1's auc: 0.843247	valid_1's binary_logloss: 0.303201
[100]	training's auc: 0.864404	training's binary_logloss: 0.285918	valid_1's auc: 0.850669	valid_1's binary_logloss: 0.297051
[150]	training's auc: 0.872021	training's binary_logloss: 0.279689	valid_1's auc: 0.852599	valid_1's binary_logloss: 0.295379
[200]	training's auc: 0.878087	training's binary_logloss: 0.274771	valid_1's auc: 0.853502	valid_1's binary_logloss: 0.294627
Early stopping, best iteration is:
[213]	training's auc: 0.879597	training's binary_logloss: 0.273571	valid_1's auc: 0.85368	valid_1's binary_logloss: 0.294479
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.852032	training's binary_logloss: 0.296658	valid_1's auc: 0.843977	valid_1's binary_logloss: 0.302616
[100]	training's auc: 0.86447	training's b

In [72]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 372
variables con importancia acumulada al 99% : 364
variables con zero importancia : 0


In [73]:
keep_columns_v45 = keep_columns

In [74]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851838	training's binary_logloss: 0.296785	valid_1's auc: 0.84296	valid_1's binary_logloss: 0.303513
[100]	training's auc: 0.864326	training's binary_logloss: 0.285975	valid_1's auc: 0.850665	valid_1's binary_logloss: 0.297237
[150]	training's auc: 0.871923	training's binary_logloss: 0.279713	valid_1's auc: 0.85279	valid_1's binary_logloss: 0.295456
[200]	training's auc: 0.878074	training's binary_logloss: 0.274762	valid_1's auc: 0.853709	valid_1's binary_logloss: 0.294749
Early stopping, best iteration is:
[221]	training's auc: 0.880382	training's binary_logloss: 0.272869	valid_1's auc: 0.853974	valid_1's binary_logloss: 0.294484
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851636	training's binary_logloss: 0.296808	valid_1's auc: 0.843137	valid_1's binary_logloss: 0.30316
[100]	training's auc: 0.864274	training's bi

In [75]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 364
variables con importancia acumulada al 99% : 356
variables con zero importancia : 0


In [ ]:
keep_columns_v46 = keep_columns

In [76]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85178	training's binary_logloss: 0.296832	valid_1's auc: 0.843617	valid_1's binary_logloss: 0.303226
[100]	training's auc: 0.864055	training's binary_logloss: 0.286107	valid_1's auc: 0.850742	valid_1's binary_logloss: 0.297183
[150]	training's auc: 0.871666	training's binary_logloss: 0.279818	valid_1's auc: 0.853121	valid_1's binary_logloss: 0.295246
[200]	training's auc: 0.877947	training's binary_logloss: 0.274812	valid_1's auc: 0.853876	valid_1's binary_logloss: 0.294567
[250]	training's auc: 0.883329	training's binary_logloss: 0.270481	valid_1's auc: 0.854343	valid_1's binary_logloss: 0.29419
[300]	training's auc: 0.888206	training's binary_logloss: 0.266472	valid_1's auc: 0.854575	valid_1's binary_logloss: 0.293962
Early stopping, best iteration is:
[291]	training's auc: 0.887341	training's binary_logloss: 0.26719	valid_1's auc: 0.854626	valid_1's binary_logloss: 0.293979
**

In [77]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 356
variables con importancia acumulada al 99% : 349
variables con zero importancia : 0


In [78]:
keep_columns_v47 = keep_columns

In [79]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851663	training's binary_logloss: 0.296874	valid_1's auc: 0.843589	valid_1's binary_logloss: 0.303179
[100]	training's auc: 0.86414	training's binary_logloss: 0.286005	valid_1's auc: 0.851046	valid_1's binary_logloss: 0.296987
[150]	training's auc: 0.871855	training's binary_logloss: 0.279737	valid_1's auc: 0.853118	valid_1's binary_logloss: 0.295259
[200]	training's auc: 0.878082	training's binary_logloss: 0.274714	valid_1's auc: 0.854069	valid_1's binary_logloss: 0.294354
[250]	training's auc: 0.883311	training's binary_logloss: 0.270432	valid_1's auc: 0.85456	valid_1's binary_logloss: 0.293943
Early stopping, best iteration is:
[263]	training's auc: 0.884555	training's binary_logloss: 0.269379	valid_1's auc: 0.854676	valid_1's binary_logloss: 0.293832
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851421	training's b

In [80]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 349
variables con importancia acumulada al 99% : 342
variables con zero importancia : 0


In [81]:
keep_columns_v48 = keep_columns

In [82]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85185	training's binary_logloss: 0.296772	valid_1's auc: 0.843546	valid_1's binary_logloss: 0.303043
[100]	training's auc: 0.864071	training's binary_logloss: 0.286117	valid_1's auc: 0.85054	valid_1's binary_logloss: 0.297163
[150]	training's auc: 0.871729	training's binary_logloss: 0.279875	valid_1's auc: 0.852542	valid_1's binary_logloss: 0.295449
[200]	training's auc: 0.87782	training's binary_logloss: 0.27498	valid_1's auc: 0.853555	valid_1's binary_logloss: 0.294577
[250]	training's auc: 0.883166	training's binary_logloss: 0.270558	valid_1's auc: 0.854058	valid_1's binary_logloss: 0.294174
Early stopping, best iteration is:
[266]	training's auc: 0.884745	training's binary_logloss: 0.26928	valid_1's auc: 0.854235	valid_1's binary_logloss: 0.294047
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851763	training's bina

In [83]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 342
variables con importancia acumulada al 99% : 335
variables con zero importancia : 0


In [84]:
keep_columns_v49 = keep_columns

In [85]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85149	training's binary_logloss: 0.296946	valid_1's auc: 0.843349	valid_1's binary_logloss: 0.303196
[100]	training's auc: 0.863845	training's binary_logloss: 0.286294	valid_1's auc: 0.850732	valid_1's binary_logloss: 0.297104
[150]	training's auc: 0.871388	training's binary_logloss: 0.280058	valid_1's auc: 0.852805	valid_1's binary_logloss: 0.295306
[200]	training's auc: 0.877529	training's binary_logloss: 0.275098	valid_1's auc: 0.853701	valid_1's binary_logloss: 0.294526
Early stopping, best iteration is:
[228]	training's auc: 0.880568	training's binary_logloss: 0.272597	valid_1's auc: 0.854	valid_1's binary_logloss: 0.294251
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851685	training's binary_logloss: 0.296829	valid_1's auc: 0.8435	valid_1's binary_logloss: 0.303055
[100]	training's auc: 0.864284	training's binar

In [86]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 335
variables con importancia acumulada al 99% : 328
variables con zero importancia : 0


In [88]:
keep_columns_v50 = keep_columns

In [87]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851528	training's binary_logloss: 0.296935	valid_1's auc: 0.842916	valid_1's binary_logloss: 0.303549
[100]	training's auc: 0.863998	training's binary_logloss: 0.286102	valid_1's auc: 0.850036	valid_1's binary_logloss: 0.297601
[150]	training's auc: 0.871577	training's binary_logloss: 0.279946	valid_1's auc: 0.852495	valid_1's binary_logloss: 0.295632
[200]	training's auc: 0.87781	training's binary_logloss: 0.27495	valid_1's auc: 0.853364	valid_1's binary_logloss: 0.294892
[250]	training's auc: 0.883286	training's binary_logloss: 0.270545	valid_1's auc: 0.853743	valid_1's binary_logloss: 0.29457
Early stopping, best iteration is:
[249]	training's auc: 0.883185	training's binary_logloss: 0.270622	valid_1's auc: 0.853769	valid_1's binary_logloss: 0.294549
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851661	training's bi

In [89]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 328
variables con importancia acumulada al 99% : 321
variables con zero importancia : 0


In [91]:
keep_columns_v51 = keep_columns

In [90]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851706	training's binary_logloss: 0.296777	valid_1's auc: 0.842972	valid_1's binary_logloss: 0.303483
[100]	training's auc: 0.864084	training's binary_logloss: 0.286091	valid_1's auc: 0.850598	valid_1's binary_logloss: 0.2973
[150]	training's auc: 0.871564	training's binary_logloss: 0.279897	valid_1's auc: 0.853184	valid_1's binary_logloss: 0.295278
[200]	training's auc: 0.877607	training's binary_logloss: 0.275004	valid_1's auc: 0.854001	valid_1's binary_logloss: 0.294495
Early stopping, best iteration is:
[237]	training's auc: 0.881587	training's binary_logloss: 0.271829	valid_1's auc: 0.854244	valid_1's binary_logloss: 0.294229
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851533	training's binary_logloss: 0.29688	valid_1's auc: 0.843212	valid_1's binary_logloss: 0.303049
[100]	training's auc: 0.864041	training's bi

In [92]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 321
variables con importancia acumulada al 99% : 315
variables con zero importancia : 0


In [93]:
keep_columns_v52 = keep_columns

In [94]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851373	training's binary_logloss: 0.296952	valid_1's auc: 0.843745	valid_1's binary_logloss: 0.303183
[100]	training's auc: 0.863703	training's binary_logloss: 0.286261	valid_1's auc: 0.850617	valid_1's binary_logloss: 0.29727
[150]	training's auc: 0.87133	training's binary_logloss: 0.280116	valid_1's auc: 0.85263	valid_1's binary_logloss: 0.295652
[200]	training's auc: 0.877351	training's binary_logloss: 0.275288	valid_1's auc: 0.85359	valid_1's binary_logloss: 0.294945
[250]	training's auc: 0.882807	training's binary_logloss: 0.270955	valid_1's auc: 0.854094	valid_1's binary_logloss: 0.294531
Early stopping, best iteration is:
[272]	training's auc: 0.884995	training's binary_logloss: 0.269168	valid_1's auc: 0.854289	valid_1's binary_logloss: 0.294395
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851504	training's bin

In [95]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 315
variables con importancia acumulada al 99% : 308
variables con zero importancia : 0


In [96]:
keep_columns_v53 = keep_columns

In [97]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851467	training's binary_logloss: 0.297028	valid_1's auc: 0.843431	valid_1's binary_logloss: 0.303376
[100]	training's auc: 0.86375	training's binary_logloss: 0.286322	valid_1's auc: 0.850711	valid_1's binary_logloss: 0.297298
[150]	training's auc: 0.8714	training's binary_logloss: 0.280096	valid_1's auc: 0.852818	valid_1's binary_logloss: 0.295441
[200]	training's auc: 0.877539	training's binary_logloss: 0.275203	valid_1's auc: 0.853316	valid_1's binary_logloss: 0.294929
Early stopping, best iteration is:
[217]	training's auc: 0.879351	training's binary_logloss: 0.273677	valid_1's auc: 0.853429	valid_1's binary_logloss: 0.294833
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851701	training's binary_logloss: 0.29688	valid_1's auc: 0.843073	valid_1's binary_logloss: 0.303301
[100]	training's auc: 0.864059	training's bin

In [98]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 308
variables con importancia acumulada al 99% : 302
variables con zero importancia : 0


In [99]:
keep_columns_v54 = keep_columns

In [100]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851437	training's binary_logloss: 0.297094	valid_1's auc: 0.843122	valid_1's binary_logloss: 0.303533
[100]	training's auc: 0.863643	training's binary_logloss: 0.286399	valid_1's auc: 0.850384	valid_1's binary_logloss: 0.297536
[150]	training's auc: 0.871476	training's binary_logloss: 0.28014	valid_1's auc: 0.852443	valid_1's binary_logloss: 0.295921
[200]	training's auc: 0.877548	training's binary_logloss: 0.275203	valid_1's auc: 0.853351	valid_1's binary_logloss: 0.295132
[250]	training's auc: 0.882956	training's binary_logloss: 0.270923	valid_1's auc: 0.853745	valid_1's binary_logloss: 0.294882
Early stopping, best iteration is:
[248]	training's auc: 0.882717	training's binary_logloss: 0.271114	valid_1's auc: 0.853806	valid_1's binary_logloss: 0.294839
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851678	training's 

In [101]:
keep_columns, zero_importance = get_feature_selection(fi)

total de variables : 302
variables con importancia acumulada al 99% : 296
variables con zero importancia : 0


In [102]:
keep_columns_v55 = keep_columns

In [103]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns], y_train, test[keep_columns])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851471	training's binary_logloss: 0.297066	valid_1's auc: 0.84318	valid_1's binary_logloss: 0.303527
[100]	training's auc: 0.863755	training's binary_logloss: 0.286345	valid_1's auc: 0.850307	valid_1's binary_logloss: 0.297483
[150]	training's auc: 0.871491	training's binary_logloss: 0.280129	valid_1's auc: 0.85248	valid_1's binary_logloss: 0.295651
[200]	training's auc: 0.877533	training's binary_logloss: 0.275264	valid_1's auc: 0.853307	valid_1's binary_logloss: 0.294951
[250]	training's auc: 0.882775	training's binary_logloss: 0.270927	valid_1's auc: 0.853637	valid_1's binary_logloss: 0.294604
Early stopping, best iteration is:
[244]	training's auc: 0.882178	training's binary_logloss: 0.271421	valid_1's auc: 0.853661	valid_1's binary_logloss: 0.294592
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.851497	training's b

#### Utilizando 397 variables

In [105]:
################# Utilizando 397 variables
########## variables con importancia acumulada al 99% : 342
len(keep_columns_v41), len(keep_columns_v48)

(397, 342)

In [106]:
[test_probs,fi] = cross_validation_lightgbm(train[keep_columns_v41], y_train, test[keep_columns_v41])

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85201	training's binary_logloss: 0.29667	valid_1's auc: 0.843466	valid_1's binary_logloss: 0.303306
[100]	training's auc: 0.864574	training's binary_logloss: 0.285876	valid_1's auc: 0.851106	valid_1's binary_logloss: 0.297129
[150]	training's auc: 0.872169	training's binary_logloss: 0.279504	valid_1's auc: 0.853527	valid_1's binary_logloss: 0.294962
Early stopping, best iteration is:
[178]	training's auc: 0.875734	training's binary_logloss: 0.27664	valid_1's auc: 0.854082	valid_1's binary_logloss: 0.294555
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.85159	training's binary_logloss: 0.296848	valid_1's auc: 0.843391	valid_1's binary_logloss: 0.303083
[100]	training's auc: 0.864355	training's binary_logloss: 0.285854	valid_1's auc: 0.850801	valid_1's binary_logloss: 0.296643
[150]	training's auc: 0.871927	training's bin

In [107]:
test_probs.name = 'target'
test_probs.to_csv('../results/lightgbm_with_397features_score_0.85347.csv') ### score de 0.85487 en la tabla publica

In [108]:
train = train[keep_columns_v41]
test = test[keep_columns_v41]

In [109]:
train.to_pickle('../data/train_features397_score_0.85487.pkl')
test.to_pickle('../data/test_features397_score_0.85487.pkl')

In [110]:
aux = (train.isna().sum()/len(train)).sort_values()
features_without_na = aux[aux==0]
print(f'feature sin missing values {len(features_without_na)} representando el {round(len(features_without_na)/len(train.columns), 3)} de features totales')
list_prec_na = [(0,0.1), (0.1,0.2), (0.2,0.3), (0.3,0.4), (0.4,0.5), (0.5,0.6), (0.6,0.7), (0.7,0.8), (0.8,0.9),(0.9,1.1)]
for i in list_prec_na:
    features_with_na = aux[(aux>=i[0]) & (aux<i[1])]
    print(f'features con porc de missing values de {i[0]} a {i[1]} : {len(features_with_na)} representando el {round(len(features_with_na)/len(train.columns), 3)} de features totales')

feature sin missing values 37 representando el 0.093 de features totales
features con porc de missing values de 0 a 0.1 : 100 representando el 0.252 de features totales
features con porc de missing values de 0.1 a 0.2 : 6 representando el 0.015 de features totales
features con porc de missing values de 0.2 a 0.3 : 1 representando el 0.003 de features totales
features con porc de missing values de 0.3 a 0.4 : 17 representando el 0.043 de features totales
features con porc de missing values de 0.4 a 0.5 : 61 representando el 0.154 de features totales
features con porc de missing values de 0.5 a 0.6 : 44 representando el 0.111 de features totales
features con porc de missing values de 0.6 a 0.7 : 15 representando el 0.038 de features totales
features con porc de missing values de 0.7 a 0.8 : 30 representando el 0.076 de features totales
features con porc de missing values de 0.8 a 0.9 : 52 representando el 0.131 de features totales
features con porc de missing values de 0.9 a 1.1 : 71 rep

In [2]:
import pandas as pd
train = pd.read_pickle('../data/train_features397_score_0.85487.pkl')
test = pd.read_pickle('../data/test_features397_score_0.85487.pkl')

In [3]:
train.to_csv('../data/train_features397_score_0.85487.csv')
test.to_csv('../data/test_features397_score_0.85487.csv')